In [8]:
import pandas as pd
from pathlib import Path

In [9]:
paths = sorted(list(Path("/home/clay/research/kaggle/kaggle_llm/data/wikipedia_pages2/").glob("*")))
df = pd.concat([
    pd.read_parquet(p)
    for p in paths
])

In [10]:
df.shape

(131049, 3)

In [14]:
topics = sorted(list(df["title"].unique()))

In [16]:
i = 2000
topics[i: i+50]

['2015 RX245',
 '2015 TB145',
 '2015 TC25',
 '2015 TH367',
 '2015 XJ351',
 '2015 in paleomammalogy',
 '2015 in paleontology',
 '2016 AZ8',
 '2016 CP31',
 '2016 DV1',
 '2016 PQ',
 '2016 United States Elizabethkingia outbreak',
 '2016 WF9',
 '2016 in Antarctica',
 '2016 in archosaur paleontology',
 '2016 in arthropod paleontology',
 '2017 AG13',
 '2017 BS5',
 '2017 DB120',
 '2017 FO161',
 '2017 MB7',
 '2017 QC36',
 '2017 QP1',
 '2017 SG33',
 '2017 SV13',
 '2017 SX17',
 '2017 VL2',
 '2017 VR12',
 '2017 VW13',
 '2017 in arthropod paleontology',
 '2017 in paleobotany',
 '2017 in paleomammalogy',
 '2017–2018 United States flu season',
 '2017–2019 world oil market chronology',
 '2018 EC4',
 '2018 FC4',
 '2018 RY7',
 '2018 VG18',
 '2018 VM35',
 '2018 heat wave',
 '2018 in amphibian paleontology',
 '2018 in arthropod paleontology',
 '2018 in echinoderm paleontology',
 '2018 in paleomammalogy',
 '2018 in paleontology',
 '2018 in spaceflight',
 '2019 AQ3',
 '2019 BE5',
 '2019 BZ3',
 '2019 EU5']

In [37]:
def begins_with_year(item):
    try:
        _ = int(item[:4])
    except:
        return False
    return len(item) > 5 and item[4] == " "


def is_year_in_something(item):
    try:
        _ = int(item[:4])
    except:
        return False
    return item[4:8] == " in "


def is_list_of_equations(item):
    return ("list of" in item) and ("equation" in item)


df[
    (
        df["title"].apply(begins_with_year)
        | df["title"].apply(is_year_in_something)
        | df["title"].apply(is_list_of_equations)
    )
]["title"]

231                           2018 RY7
347                           2018 FC4
757                         2000 SG344
945                      1458 (number)
1862     1884 Baltimore Orioles season
                     ...              
24727                       2002 RN109
24774                          2014 RC
24801             3102 aluminium alloy
24806                         2014 XL7
24922                        2013 RF98
Name: title, Length: 334, dtype: object

In [33]:
df.set_index("title").loc["Non-blanching rash"]["text"]

'A non-blanching rash (NBR) is a skin rash that does not fade when pressed with, and viewed through, a glass.\n\nIt is a characteristic of both purpuric and petechial rashes. Individual purpura measure 3–10\xa0mm (0.3–1\xa0cm, -\xa0in), whereas petechiae measure less than 3\xa0mm.\n\nA non-blanching rash can be a symptom of bacterial meningitis, but this is not the exclusive cause.\n\nReferences\n\nExternal links \n\nCutaneous conditions'

In [34]:
df.set_index("title").loc["Gradient-enhanced kriging"]["text"]

"Gradient-enhanced kriging (GEK) is a surrogate modeling technique used in engineering. A surrogate model (alternatively known as a metamodel, response surface or emulator) is a prediction of the output of an expensive computer code. \nThis prediction is based on a small number of evaluations of the expensive computer code.\n\nIntroduction \n\nAdjoint solvers are now becoming available in a range of computational fluid dynamics (CFD) solvers, such as Fluent, OpenFOAM, SU2 and US3D. Originally developed for optimization, adjoint solvers are now finding more and more use in uncertainty quantification.\n\nLinear speedup \nAn adjoint solver allows one to compute the gradient of the quantity of interest with respect to all design parameters at the cost of one additional solve. This, potentially, leads to a linear speedup: the computational cost of constructing an accurate surrogate decrease, and the resulting computational speedup  scales linearly with the number  of design parameters.\n\nT